# Project Requirements 

## Firstly, you will required to submit the following:

A description of the problem and a discussion of the background. 

A description of the data and how it will be used to solve the problem. 

## Secondly, the final deliverables of the project will be:

A link to your Notebook on your Github repository, showing your code. 

A full report consisting of all of the following components:

Introduction where you discuss the business problem and who would be interested in this project.

Data where you describe the data that will be used to solve the problem and the source of the data.

Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any 
inferential statistical testing that you performed, if any, and what machine learnings were used and why.

Results section where you discuss the results.

Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.

Conclusion section where you conclude the report.

### Business Problem and Background:

I’m really enthusiastic to start a business, mostly interested in Restaurants, coffee shop or preschool. For IBM coursera capstone project I want to use my skills to predict the recommended location to open coffee shop in Cleveland, Ohio.
Cleveland is a beautiful city in Ohio with nice and kind people. It has decent number of restaurants and coffee shops. In this analysis trying to find location for successful business. Even tough it is stressed out city in US, I believe there is good chances to run a successful coffee shop.

#### Coffee is culture
Going back in time, we’ve seen that coffee is not just a drink. It is actually a lot more. In the past, people gathered in houses to share a cup of coffee and exchange information.

In North America and a lot of western European countries, Starbucks and other coffee chains have dominated the market because they are providing to the customers a complete experience around coffee. Coffee and friends, coffee and work, coffee and snacks and coffee to go! The Irish mix coffee with whiskey and they call it “Irish coffee”. In Italy the espresso and espresso machine were born and made it the favorite whole-day beverage of Italians. In Greece, there is the “kafenio”, an old fashioned coffee shop for old gentlemen, where they drink Greek coffee and exchange political ideas or they play cards and a board game called “tavli”. Then also, in some countries, like Colombia or Brazil, the whole economy is based in coffee.

So this makes us really curious to know more interesting facts about coffee as a commodity itself but also as a beverage.

For more info please visit: https://www.agiboo.com/16-interesting-facts-about-coffee/

#### Audience

Business investors who want to start a coffee shop in particular area Cleveland,Ohio.

Novice people who has interest in exploring the data realted insights for starting their own business. 


### Data Selection

The data is collected from foursquare API about coffee shops in Clelveand, Ohio and visulaize the clean dataframe for recommending the ideal location.

### Methodology

Here steps used in analysis and modeling the project:

1. Importing libraries needed.
2. Setting up credentials for foursquare to fetch the data and all necessary steps to get data.
3. Converting json to dataframe and cleaning the data 
4. Visulaizing the cofee shops using folium
5. Apply K-means clustereing.
6. Recommendation/Conclusion based on analysis.

### References

https://labs.cognitiveclass.ai/tools/jupyterlab/lab/tree/labs/DP0701EN/DP0701EN-3-3-2-Neighborhoods-New-York-py-v1.0.ipynb
https://stackoverflow.com/questions/48085110/no-module-named-folium-plugins-python-3-6
https://medium.com/@aboutiana/a-brief-guide-to-using-foursquare-api-with-a-hands-on-example-on-python-6fc4d5451203
https://developer.foursquare.com/
https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf
https://dev.to/chijade/capstone-project-the-battle-of-neighborhoods-4ha3





In [30]:
# Libraries needed are imported here

import sys
import numpy as np
import requests # library for handle requests
import pandas as pd # library for data analsysis
!pip install folium
import folium # library for map rendering
import json
import io
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim #location to latitude and longtitude
from folium import plugins

print('Libraries Imported')

Libraries Imported


In [6]:
# Setting up foursquare credentials to fetch data 
CLIENT_ID = 'N4GIWTCJ0DPW3ST5P0EEVB25W0JHGW5ZTBJZKDKUBFTJGOK0' # your Foursquare ID
CLIENT_SECRET = 'KRLYG4052BTLWMRWK445JLY10SHUBRLCOBRMAQLO1RRZ1PF0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: N4GIWTCJ0DPW3ST5P0EEVB25W0JHGW5ZTBJZKDKUBFTJGOK0
CLIENT_SECRET:KRLYG4052BTLWMRWK445JLY10SHUBRLCOBRMAQLO1RRZ1PF0


In [7]:
# Parameters 
request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Cleveland",
    "radius": 1000,
    "limit": 25}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [12]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [15]:
# Checking cleveland city latitude and longitude and also all others values
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"], d["suggestedBounds"], d["geocode"]

# In geocode we can examine more details about the location like state, country, latitude, longitude etc.

('city',
 'Cleveland',
 'Cleveland',
 {'ne': {'lat': 41.5041843172057, 'lng': -81.68390763895238},
  'sw': {'lat': 41.49608830723969, 'lng': -81.69999756762437}},
 {'what': '',
  'where': 'cleveland',
  'center': {'lat': 41.4995, 'lng': -81.69541},
  'displayString': 'Cleveland, OH, United States',
  'cc': 'US',
  'geometry': {'bounds': {'ne': {'lat': 41.604436, 'lng': -81.532744},
    'sw': {'lat': 41.390628, 'lng': -81.878984}}},
  'slug': 'cleveland-ohio',
  'longId': '72057594043078465'})

In [16]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [17]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [18]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 25


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5773cc60498e63c34d8c774c',
  'name': 'REBoL',
  'location': {'address': '101 W Superior Ave',
   'lat': 41.4991,
   'lng': -81.69382,
   'labeledLatLngs': [{'label': 'display', 'lat': 41.4991, 'lng': -81.69382}],
   'postalCode': '44113',
   'cc': 'US',
   'city': 'Cleveland',
   'state': 'OH',
   'country': 'United States',
   'formattedAddress': ['101 W Superior Ave',
    'Cleveland, OH 44113',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d16d941735',
    'name': 'Café',
    'pluralName': 'Cafés',
    'shortName': 'Café',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-5773cc60498e63c34d8c774c-0'}

In [19]:
#examine the items
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '59f39077a423626e09befb94',
  'name': 'Phoenix Coffee',
  'location': {'address': '826 W. St. Clair Ave',
   'lat': 41.49893605763097,
   'lng': -81.69926620723018,
   'labeledLatLngs': [{'label': 'display',
     'lat': 41.49893605763097,
     'lng': -81.69926620723018}],
   'postalCode': '44113',
   'cc': 'US',
   'neighborhood': 'Downtown Cleveland',
   'city': 'Cleveland',
   'state': 'OH',
   'country': 'United States',
   'formattedAddress': ['826 W. St. Clair Ave',
    'Cleveland, OH 44113',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}}

In [23]:
#json to dataframe and cleaning the data
df_raw = []
for item in items:
    venue = item["venue"]
    categories, id, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (id, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["id", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()
df.tail()
df.dtypes
df.info()
df.shape

{'address': '101 W Superior Ave', 'lat': 41.4991, 'lng': -81.69382, 'labeledLatLngs': [{'label': 'display', 'lat': 41.4991, 'lng': -81.69382}], 'postalCode': '44113', 'cc': 'US', 'city': 'Cleveland', 'state': 'OH', 'country': 'United States', 'formattedAddress': ['101 W Superior Ave', 'Cleveland, OH 44113', 'United States']}
{'address': '826 W. St. Clair Ave', 'lat': 41.49893605763097, 'lng': -81.69926620723018, 'labeledLatLngs': [{'label': 'display', 'lat': 41.49893605763097, 'lng': -81.69926620723018}], 'postalCode': '44113', 'cc': 'US', 'neighborhood': 'Downtown Cleveland', 'city': 'Cleveland', 'state': 'OH', 'country': 'United States', 'formattedAddress': ['826 W. St. Clair Ave', 'Cleveland, OH 44113', 'United States']}
{'address': '412 Superior Avenue', 'lat': 41.5008484975666, 'lng': -81.69121622267214, 'labeledLatLngs': [{'label': 'display', 'lat': 41.5008484975666, 'lng': -81.69121622267214}], 'cc': 'US', 'city': 'Cleveland', 'state': 'OH', 'country': 'United States', 'formatte

(24, 7)

In [27]:
cleveland_coordinates = d["geocode"]["center"]
cleveland_coordinates

{'lat': 41.4995, 'lng': -81.69541}

In [31]:
#visulaizing the data of coffee shops in cleveland 
cle_map = folium.Map(location=[41.4995, -81.69541], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='green',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(cle_map)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
cle_map.add_child(plugins.HeatMap(hm_data))

cle_map

In [36]:
latitude = 41.4995 
longitude= -81.69541

map_cleveland = folium.Map(location=[latitude, longitude], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="O Day Roi!",
    color='purple',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_cleveland)
map_cleveland

# We can notice here that the East 6th street near Superior avenue east is the recommended location for coffee shop.

In [39]:
df

,id,name,shortname,address,postalcode,lat,lng
0,5773cc60498e63c34d8c774c,REBoL,Café,,44113,41.499100,-81.693820
1,59f39077a423626e09befb94,Phoenix Coffee,Coffee Shop,,44113,41.498936,-81.699266
2,527163b411d2db2dcd46ce48,Pour Cleveland,Coffee Shop,,44115,41.499750,-81.689148
3,4ae07054f964a520497f21e3,Phoenix Coffee,Coffee Shop,,44114,41.501825,-81.688294
4,4be5ab7b2468c92896580043,Titans Gym,Gym / Fitness,,44115,41.498777,-81.688768
5,55e5bd63498e5e942f82cde3,Starbucks,Coffee Shop,,44113,41.497146,-81.693850
6,4ae845eef964a5206bae21e3,Erie Island Coffee Company,Coffee Shop,,44115,41.498862,-81.690071
7,5273debf11d21f1806525f80,The Tea Lab,Tea Room,,44115,41.499310,-81.689122
8,582dfc410b56560501afc30a,Starbucks,Coffee Shop,,44114,41.503408,-81.691376
9,4d30873682fd5481c227b4a8,Oasis Urban Cafe,Café,,44114,41.503816,-81.690036


In [41]:
print('There are {} uniques categories.'.format(len(df['id'].unique())))

There are 24 uniques categories.


In [42]:
# one hot encoding
onehot = pd.get_dummies(df[['name']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['name'] = df['name'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,name,AJ Rocco's,Au Bon Pain,Court House Cafe,Dunkin',Erie Island Coffee Company,Good To Go Cafe,Joe Maxx Coffee Company Public Sq,Noshery Marketplace,Nourish Cafe,Oasis Urban Cafe,Phoenix Coffee,Pour Cleveland,REBoL,Starbucks,The Copper Moon,The Espresso Room,The Tea Lab,Titans Gym
0,REBoL,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,Phoenix Coffee,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Pour Cleveland,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Phoenix Coffee,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Titans Gym,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [43]:
onehot.shape

(24, 19)

In [44]:
grouped = onehot.groupby('name').mean().reset_index()
grouped

,name,AJ Rocco's,Au Bon Pain,Court House Cafe,Dunkin',Erie Island Coffee Company,Good To Go Cafe,Joe Maxx Coffee Company Public Sq,Noshery Marketplace,Nourish Cafe,Oasis Urban Cafe,Phoenix Coffee,Pour Cleveland,REBoL,Starbucks,The Copper Moon,The Espresso Room,The Tea Lab,Titans Gym
0,AJ Rocco's,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Au Bon Pain,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Court House Cafe,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Dunkin',0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Erie Island Coffee Company,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Good To Go Cafe,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
6,Joe Maxx Coffee Company Public Sq,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7,Noshery Marketplace,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
8,Nourish Cafe,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9,Oasis Urban Cafe,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [45]:
grouped.shape

(18, 19)

In [46]:
# set number of clusters
kclusters = 4

clustering = grouped.drop('name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 3, 0, 0, 0, 0, 0], dtype=int32)

In [49]:
# add clustering labels
grouped.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [51]:
grouped

,Cluster Labels,name,AJ Rocco's,Au Bon Pain,Court House Cafe,Dunkin',Erie Island Coffee Company,Good To Go Cafe,Joe Maxx Coffee Company Public Sq,Noshery Marketplace,Nourish Cafe,Oasis Urban Cafe,Phoenix Coffee,Pour Cleveland,REBoL,Starbucks,The Copper Moon,The Espresso Room,The Tea Lab,Titans Gym
0,0,AJ Rocco's,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,Au Bon Pain,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,Court House Cafe,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,Dunkin',0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3,Erie Island Coffee Company,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,Good To Go Cafe,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
6,0,Joe Maxx Coffee Company Public Sq,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7,0,Noshery Marketplace,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
8,0,Nourish Cafe,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9,0,Oasis Urban Cafe,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
# As said before, We can notice here that the East 6th street near Superior avenue east is the recommended location for coffee shop. It is also clearly good suggetion since we can see shopping mall, library and some banks near the location where we can expect more business.
#Futher more in future I may plan to extend this project using data about cleveland neighborhood and foursquare API data to predict more insights and perform clustering also.

